In [6]:
import os, sys, time, json, contextlib, io
from importlib import reload

from m.fileio import MediaFile
from m.ui import *
from m.util import *
from m.encode import *

testfile = 'caches/fdumpster.mp4'
testfile2 = 'caches/tallcat.gif'
testfile3 = 'caches/nestest.nes'
testfile4 = 'caches/GtKSwIy.jpeg'

mf = MediaFile(testfile)

if mf.e:
    print("There were errors when opening the file.")
    print(mf.e)
    if mf.stderr:
        print(mf.stderr)
else:
    pass
    #Run tests
    pass








Finished running


In [7]:
from importlib import reload

import m.encode as enc
reload(enc)

loadedparams = enc.EncodeParams("M2G4")
print(loadedparams)
print(loadedparams.pal1bw)
print(loadedparams.pal2gs)
print(loadedparams.pal4gs)
print(loadedparams.pal4col)


[0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 255, 255, 0, 0, 0, 255, 2

In [10]:
# Hero Wars skin costs calculator. Calculates minimum number of skins stones
# to spend to reach a certain cumulative level, configurable below.
# Use case: Game wants you to increase 150 skin levels. Can you actually
# accomplish this task with the resources you currently have?
import itertools
otherskincost = [5000] + [65]*4 + [105]*5 + [225]*5 + [300]*5 + [700]*5 + [790]*5 + [1050]*5 + [1160]*5 + [1270]*5 + [1370]*5 + [1460]*5 + [1570]*5

cost = otherskincost
cumsum = list(itertools.accumulate(cost))
levels_needed = 146
minimal = float('inf')
minstats = []

for levelchunk in range(59):
    lvlneed = levels_needed
    levelused = 0
    skins = 0
    costs = list()
    while lvlneed:
        skins += 1
        if lvlneed > levelchunk:
            lvlneed -= levelchunk+1
            levelused = levelchunk+1
        else:
            levelused = lvlneed
            lvlneed = 0
        costs.append(cumsum[levelused])
    totalcosts = sum(costs)
    if totalcosts < minimal:
        minimal = totalcosts
        minstats = [levels_needed, levelchunk, totalcosts, skins]
    #print(f"To reach {levels_needed} levels at {levelchunk}: {sum(costs)}")

print(f"Minimal to reach level {minstats[0]} is at {minstats[3]} skins at level {minstats[1]}, costing {minstats[2]}")


Minimal to reach level 146 is at 8 skins at level 18, costing 65555
